# Либы

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

import psycopg2 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import os

load_dotenv()

True

# Подключение

In [3]:
USER = os.environ.get('POSTGRES_USER')
PASSWORD = os.environ.get('POSTGRES_PASSWORD')
DB = os.environ.get('POSTGRES_DB_stepik')

conn_string = psycopg2.connect(f"dbname={DB} user='{USER}' host='localhost' password='{PASSWORD}'")
cursor = conn_string.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()
print(f"Подключение к {record} выполнено")

Подключение к ('PostgreSQL 15.8 (Debian 15.8-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) выполнено


# 1.2 Выборка данных

In [64]:
# 1. Вывести информацию о всех книгах, хранящихся на складе.
q1 = ('  select * from book')
# 2. Выбрать названия книг и авторов из таблицы book, для поля title задать имя(псевдоним) Название, для поля author –  Автор.
q2 = ('  select title as Название, author as Автор from book')
# 3. Для упаковки каждой книги требуется один лист бумаги, цена которого 1 рубль 65 копеек. Посчитать стоимость упаковки для каждой книги 
# (сколько денег потребуется, чтобы упаковать все экземпляры книги). В запросе вывести название книги, ее количество и стоимость упаковки, 
# последний столбец назвать pack. 
q3 = (' select title, amount, amount * 1.65 as pack \
        from book')
# 4. В конце года цену каждой книги на складе пересчитывают – снижают ее на 30%. Написать SQL запрос, который из таблицы book выбирает названия, авторов, количества и 
# вычисляет новые цены книг. Столбец с новой ценой назвать new_price, цену округлить до 2-х знаков после запятой.
q4 = (' select title, author, amount, \
        round((price*0.7),2) as new_price \
        from book')
# 5. При анализе продаж книг выяснилось, что наибольшей популярностью пользуются книги Михаила Булгакова, на втором месте книги Сергея Есенина. 
# Исходя из этого решили поднять цену книг Булгакова на 10%, а цену книг Есенина - на 5%. Написать запрос, куда включить автора, название книги и новую цену, 
# последний столбец назвать new_price. Значение округлить до двух знаков после запятой.
q5 = (  'select author, title, '
        'round( '
        '       case '
        '           when author = \'Булгаков М.А.\' then price * 1.1 '
        '           when author = \'Есенин С.А.\' then price * 1.05 '
        '           else price '
        '       end, 2) as new_price '
        'from book')
# 6. Вывести автора, название  и цены тех книг, количество которых меньше 10.
q6 = (' select author, title, price \
        from book where amount < 10')
# 7. Вывести название, автора,  цену  и количество всех книг, цена которых меньше 500 или больше 600, 
# а стоимость всех экземпляров этих книг больше или равна 5000.
q7 = (' select title, author, price, amount \
        from book \
        where (price < 500 OR price > 600) and price*amount >= 5000')
# 8. Вывести название и авторов тех книг, цены которых принадлежат интервалу от 540.50 до 800 (включая границы),  
# а количество или 2, или 3, или 5, или 7.
q8 = (' select title, author \
        from book \
        where price between 540.5 and 800 and amount in (2, 3, 5, 7)')
# 9. Вывести  автора и название  книг, количество которых принадлежит интервалу от 2 до 14 (включая границы). 
# Информацию  отсортировать сначала по авторам (в обратном алфавитном порядке), а затем по названиям книг (по алфавиту).
q9 = (' select author, title \
        from book \
        where amount between 2 and 14 \
        order by author desc, title asc')
# 10. Вывести название и автора тех книг, название которых состоит из двух и более слов, а инициалы автора содержат букву «С». 
# Считать, что в названии слова отделяются друг от друга пробелами и не содержат знаков препинания, между фамилией автора 
# и инициалами обязателен пробел, инициалы записываются без пробела в формате: буква, точка, буква, точка. 
# Информацию отсортировать по названию книги в алфавитном порядке.
q10 = ('select title, author from book \
        where title like \'_% _%\' and author like \'%С.%\' \
        order by title asc')
# 11. Магазин счёл, что классика уже не пользуется популярностью, поэтому необходимо в выборке:
# Сменить всех авторов на "Донцова Дарья".
# К названию каждой книги в начале дописать "Евлампия Романова и".
# Цену поднять на 42%.
# Отсортировать по убыванию цены и убыванию названия.
q11 = ( 'select \'Донцова Дарья\' as author,' + 
        'concat(\'Евлампия Романова и \' , title) as title, ' +
        'round((price * 1.42),2) as price '+
        'from book order by price desc, title desc')

data = pd.read_sql(q11, con=conn_string)
data

,author,title,price
0,Донцова Дарья,Евлампия Романова и Братья Карамазовы,1134.59
1,Донцова Дарья,Евлампия Романова и Мастер и Маргарита,952.81
2,Донцова Дарья,Евлампия Романова и Стихотворения и поэмы,923.00
3,Донцова Дарья,Евлампия Романова и Белая гвардия,767.51
4,Донцова Дарья,Евлампия Романова и Игрок,682.31
5,Донцова Дарья,Евлампия Романова и Идиот,653.20


# 1.3 Запросы, групповые операции

In [54]:
# 1. Отобрать различные (уникальные) элементы столбца amount таблицы book.
q1 = ( 'SELECT amount FROM book GROUP BY amount')
# 2. Посчитать, количество различных книг и количество экземпляров книг каждого автора , хранящихся на складе.  
# Столбцы назвать Автор, Различных_книг и Количество_экземпляров соответственно.
q2 = (  'SELECT author AS Автор, \
         COUNT(author) AS Различных_книг, \
         SUM(amount) AS Количество_экземпляров \
         FROM book \
         GROUP BY author')
# q3 = Вывести фамилию и инициалы автора, минимальную, максимальную и среднюю цену книг каждого автора . 
# Вычисляемые столбцы назвать Минимальная_цена, Максимальная_цена и Средняя_цена соответственно.
q3 = (  'SELECT author,     MIN(price) AS Минимальная_цена, \
                            MAX(price) AS Максимальная_цена, \
                            AVG(price) AS Средняя_цена \
        FROM book \
        GROUP BY author')
# 4. Для каждого автора вычислить суммарную стоимость книг S (имя столбца Стоимость), а также вычислить налог на добавленную стоимость  
# для полученных сумм (имя столбца НДС ) , который включен в стоимость и составляет 18% (k=18),  а также стоимость книг  (Стоимость_без_НДС) 
# без него. Значения округлить до двух знаков после запятой. В запросе для расчета НДС(tax)  и Стоимости без НДС(S_without_tax)
q4 = (  'SELECT author, '
        'SUM(price * amount) AS Стоимость, '
        'ROUND((SUM(price * amount) * 0.18) / 1.18, 2) AS НДС, '
        'ROUND(SUM(price * amount) / 1.18, 2) AS Стоимость_без_НДС '
        'FROM book '
        'GROUP BY author ORDER BY author')
# 5. Вывести цену самой дешевой книги, цену самой дорогой и среднюю цену всех книг на складе. 
# Названия столбцов Минимальная_цена, Максимальная_цена, Средняя_цена соответственно. 
# Среднюю цену округлить до двух знаков после запятой. 
q5 = (' SELECT ROUND(MIN(price),2) AS Минимальная_цена, \
        ROUND(MAX(price),2) AS Максимальная_цена, \
        ROUND(AVG(price),2) AS Средняя_цена \
        FROM book')
# 6. Вычислить среднюю цену и суммарную стоимость тех книг, количество экземпляров которых принадлежит интервалу от 5 до 14, включительно. 
# Столбцы назвать Средняя_цена и Стоимость, значения округлить до 2-х знаков после запятой.
q6 = (' SELECT ROUND(AVG(price),2) AS Средняя_цена, round(SUM(price*amount),2) AS Стоимость \
        FROM book \
        WHERE amount BETWEEN 5 AND 14')
# 7. Посчитать стоимость всех экземпляров каждого автора без учета книг «Идиот» и «Белая гвардия». 
# В результат включить только тех авторов, у которых суммарная стоимость книг (без учета книг «Идиот» и «Белая гвардия») более 5000 руб. 
# Вычисляемый столбец назвать Стоимость. Результат отсортировать по убыванию стоимости.
q7 = (' SELECT author, SUM(price*amount) AS Стоимость \
        FROM book \
        WHERE title != \'Идиот\' AND title != \'Белая гвардия\' \
        GROUP BY author \
        HAVING SUM(price*amount) > 5000 \
        ORDER BY SUM(price*amount) DESC;')
#8. Объединить книги в разделы по первой букве названия.
#Каждый раздел начинать со строки, в которой непустой является только колонка 'Буква' - первая буква названия.
#Для строк с названиями книг колонка 'Буква' - пустая.
#Упорядочить разделы и названия книг внутри разделов (а также авторов для одинаковых названий) по алфавиту.
#Вывести колонки Буква, Название и Автор.
q8 = (
    'SELECT SUBSTR(title, 1, 1) AS Буква, \'\' AS Название, \'\' AS Автор '
    'FROM book '
    'WHERE title LIKE \'Б%\' '
    'GROUP BY SUBSTR(title, 1, 1) '
    'UNION '
    'SELECT \'\' AS Буква, title AS Название, author AS Автор '
    'FROM book '
    'WHERE title LIKE \'Б%\' '
    'UNION '
    'SELECT SUBSTR(title, 1, 1) AS Буква, \'\' AS Название, \'\' AS Автор '
    'FROM book '
    'WHERE title LIKE \'И%\' '
    'GROUP BY SUBSTR(title, 1, 1) '
    'UNION '
    'SELECT \'\' AS Буква, title AS Название, author AS Автор '
    'FROM book '
    'WHERE title LIKE \'И%\' '
    'UNION '
    'SELECT SUBSTR(title, 1, 1) AS Буква, \'\' AS Название, \'\' AS Автор '
    'FROM book '
    'WHERE title LIKE \'М%\' '
    'GROUP BY SUBSTR(title, 1, 1) '
    'UNION '
    'SELECT \'\' AS Буква, title AS Название, author AS Автор '
    'FROM book '
    'WHERE title LIKE \'М%\' '
    'UNION '
    'SELECT SUBSTR(title, 1, 1) AS Буква, \'\' AS Название, \'\' AS Автор '
    'FROM book '
    'WHERE title LIKE \'С%\' '
    'GROUP BY SUBSTR(title, 1, 1) '
    'UNION '
    'SELECT \'\' AS Буква, title AS Название, author AS Автор '
    'FROM book '
    'WHERE title LIKE \'С%\' '
    'ORDER BY Буква, Название, Автор'
)


data = pd.read_sql(q8, 
                   con=conn_string)
data

,Буква,Название,Автор
0,,Белая гвардия,Булгаков М.А.
1,,Братья Карамазовы,Достоевский Ф.М.
2,,Игрок,Достоевский Ф.М.
3,,Идиот,Достоевский Ф.М.
4,,Мастер и Маргарита,Булгаков М.А.
5,,Стихотворения и поэмы,Есенин С.А.
6,Б,,
7,И,,
8,М,,
9,С,,
